# Quick check data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
data_dir = "data"
receipts_csv_dir = os.path.join(data_dir, "receipts", "csv")
receipts_ocr_dir = os.path.join(data_dir, "receipts", "ocr")
images_dir = os.path.join(data_dir, "receipts", "images")

In [ ]:
receipt_list = [f for f in os.listdir(receipts_ocr_dir) if f.endswith(".csv")]
receipt_list

In [ ]:
all_receipts = pd.concat(
    [pd.read_csv(os.path.join(receipts_csv_dir, f), index_col=0, parse_dates=["Date"]) for f in receipt_list]
)
all_receipts.head()

In [ ]:
global_infos = all_receipts.groupby("original_filename")[["Merchant_Name", "Total_Amount", "Date"]].first()

In [ ]:

sns.barplot(global_infos, x="Date", y="Total_Amount", hue="Merchant_Name")

In [ ]:
global_infos.Date.iloc[0]

In [ ]:
all_amounts = pd.read_csv(os.path.join(data_dir, "Comptes.csv"), header=0, parse_dates=["Date",])

In [ ]:
all_amounts["Prix"] = all_amounts.Prix.apply(lambda x: float(x.replace(",", ".")) if isinstance(x, str) else x)
all_amounts["Total panier"] = all_amounts["Total panier"].apply(lambda x: float(x.replace(",", ".")) if isinstance(x, str) else x)
all_amounts["Prix unitaire/kg"] = all_amounts["Prix unitaire/kg"].apply(lambda x: float(x.replace("/kg", "").replace(",", ".")) if isinstance(x, str) else x)

In [ ]:
all_amounts.columns

In [ ]:
all_amounts[['Date', 'Marchand', 'Adresse', 'Article',
       'Catégorie 1', 'Catégorie 2', 'Prix']].groupby("Catégorie 1")["Prix"].sum().sort_values()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
sns.boxplot(data=all_amounts.sort_values(by="Catégorie 2"), hue="Catégorie 2", y="Catégorie 2", x="Prix", legend=False, orient='h')

In [ ]:
one_receipt = pd.read_csv(os.path.join(receipts_ocr_dir, "9bfbc8d7-b21c-48d9-8e74-41cff355b464.csv"))
one_receipt.head()

In [ ]:
one_receipt.columns

In [ ]:
disabled = ['Merchant_Address', 'currency', 'Merchant_Name', 'Total_Amount', 'Date', 'Price', 'Description', 'Line_Amount', 'Quantity']

In [ ]:
image_file = os.path.join(images_dir, one_receipt.iloc[0].original_filename)
image_file

In [ ]:
one_receipt["Quantity"].fillna(1)

In [ ]:
from constants import GLOBAL_INFO_COLUMNS
first_row = one_receipt.iloc[0]
print("\n".join([f"{k}:{v}" for k, v in first_row[GLOBAL_INFO_COLUMNS].items()]))


In [ ]:
df = first_row[GLOBAL_INFO_COLUMNS].copy()
df.name = "Ticket information"
d

# OCR API

In [ ]:
import os
import glob
from constants import IMG_DIR, OCR_DIR

In [ ]:

image_file_list = os.listdir(IMG_DIR)
image_file_list[-2:]

In [ ]:
import requests
import configparser
import pathlib

def get_nanonet_key():
    config = configparser.ConfigParser()
    config.read(os.path.join(pathlib.Path().home(), ".nanonet"))
    return config["account"]["apikey"]

In [ ]:
import json

def nanonet_ocr(image_file):
    url = "https://app.nanonets.com/api/v2/OCR/Model/9bfbc8d7-b21c-48d9-8e74-41cff355b464/LabelFile/?async=false"

    data = {"file": open(image_file, "rb")}

    response = requests.post(
        url, auth=requests.auth.HTTPBasicAuth(get_nanonet_key(), ""), files=data
    )
    return response.json()["result"]

def run_single_image_ocr(image_filename):
    image_path = os.path.join(IMG_DIR, image_filename)
    json_path = os.path.join(OCR_DIR, image_filename.replace(".jpg", ".json"))
    nanonet_result = None
    if not os.path.exists(json_path):
        print(f"{image_filename}: Querying Nanonet")
        nanonet_result = nanonet_ocr((image_path))
        with open(json_path, "w") as f:
            f.write(json.dumps(nanonet_result))
    else:
        print(f"{image_filename}: Reading cache")
        with open(json_path, "r") as f:
            nanonet_result = json.loads(f.read())
    assert nanonet_result
    ocr_df = extract_as_clean_csv(nanonet_result[0])
    return ocr_df

In [ ]:
import concurrent.futures

def run_ocr_and_save(image_filename):
    res = run_single_image_ocr(image_filename)
    ocr_file = os.path.join(OCR_DIR, image_filename.replace(".jpg", ".csv"))
    res.to_csv(ocr_file)
    return ocr_file

output_files = []
# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(run_ocr_and_save, img_file): img_file for img_file in image_file_list}
    for future in concurrent.futures.as_completed(future_to_url):
        img_file = future_to_url[future]
        try:
            data = future.result()
            output_files.append(data)
        except Exception as exc:
            print(f'{img_file} generated an exception: {exc}')
        else:
            print(f'{img_file} OCR result saved to {data}')

In [ ]:

def nanonet_ocr_multi(image_file_list):
    url = "https://app.nanonets.com/api/v2/OCR/Model/9bfbc8d7-b21c-48d9-8e74-41cff355b464/LabelFile/?async=false"

    data = {"file": [open(image_file, "rb") for image_file in image_file_list]}

    response = requests.post(
        url, auth=requests.auth.HTTPBasicAuth(get_nanonet_key(), ""), files=data
    )
    return response.json()["result"]


In [ ]:
raw_multi_response = nanonet_ocr_multi(image_file_list=image_file_list[-2:])

In [ ]:

url = 'https://app.nanonets.com/api/v2/OCR/Model/9bfbc8d7-b21c-48d9-8e74-41cff355b464/LabelFile/?async=false'

data = {'file': open(image_file, 'rb')}

response = requests.post(url, auth=requests.auth.HTTPBasicAuth(get_nanonet_key(), ''), files=data)

print(response.text)
        

In [ ]:
response.json()["result"][0]["input"]

In [ ]:
import json

with open(os.path.join(receipts_ocr_dir, "PXL_20240102_192728736.jpg".replace("jpg", "json")), "w") as f:
    f.write(json.dumps(response.json()["result"]))

In [ ]:

with open(os.path.join(receipts_ocr_dir, "PXL_20240105_181057959.jpg".replace("jpg", "json")), "r") as f:
    loaded_json = json.loads(f.read())

In [ ]:
import glob

receipt_list = glob.glob(os.path.join(receipts_ocr_dir, "*.csv"))

In [ ]:
[f for f in os.listdir(receipts_ocr_dir) if f.endswith(".csv")]

In [ ]:
loaded_json[0]["prediction"]

In [ ]:
raw_output = pd.DataFrame(response.json()["result"][0]["prediction"])
raw_output

In [ ]:
def extract_as_clean_csv(nanonet_result):
    raw_output = pd.DataFrame(nanonet_result["prediction"])
    columns_global =  ['currency', 'Merchant_Name', 'Merchant_Phone', 'Merchant_Address',
       'Total_Amount', 'Tax_Amount', 'Date',]
    processed_output1 = pd.DataFrame(columns=raw_output["label"].values, data=[raw_output["ocr_text"].values])
    processed_output = pd.DataFrame(columns=columns_global)
    for c in columns_global:
        if c in processed_output1.columns:
            processed_output[c] = processed_output1[c]
    
    raw_table = pd.DataFrame(raw_output[raw_output.label == "table"].iloc[0].cells)
    processed_table = pd.DataFrame([
        pd.Series(index=row.label, data=row["text"].values, name=i) for i, row in raw_table.groupby("row")
    ]
    )
    processed_table[columns_global] = pd.concat([processed_output for _ in range(len(processed_table))], ignore_index=True)

    processed_table["Quantity"] = processed_table["Quantity"].fillna(1)
    processed_table["Line_Amount"] = processed_table["Line_Amount"].apply(lambda x: x.replace("€", "").strip()  if isinstance(x, str) else x)
    processed_table["Price"] = processed_table["Price"].apply(lambda x: x.replace("/kg", "").replace("/ kg", "").replace("€", "").strip()  if isinstance(x, str) else x)
    processed_table["original_filename"] = response.json()["result"][0]["input"]
    return processed_table

In [ ]:
ddf = extract_as_clean_csv(response.json()["result"][0])

In [ ]:
ddf["Category"] = "" 
ddf

In [ ]:
df["Merchant_Name"] = "meh"

In [ ]:
dx = pd.Series(df)
dx

In [ ]:
dx.T.iloc[0]

In [ ]:
ddf[GLOBAL_INFO_COLUMNS] = dx
ddf

In [ ]:
raw_output["label"].values

In [ ]:
columns_global =  ['currency', 'Merchant_Name', 'Merchant_Phone', 'Merchant_Address',
       'Total_Amount', 'Tax_Amount', 'Date',]

In [ ]:

processed_output1 = pd.DataFrame(columns=raw_output["label"].values, data=[raw_output["ocr_text"].values])
processed_output = pd.DataFrame(columns=columns_global)
for c in columns_global:
    if c in processed_output1.columns:
        processed_output[c] = processed_output1[c]
processed_output

In [ ]:
len(raw_output)

In [ ]:
raw_table = pd.DataFrame(raw_output[raw_output.label == "table"].iloc[0].cells)
raw_table.head()

In [ ]:
one_receipt.columns

In [ ]:
processed_table = pd.DataFrame([
pd.Series(index=row.label, data=row["text"].values, name=i) for i, row in raw_table.groupby("row")
]
)
processed_table[columns_global] = pd.concat([processed_output for _ in range(len(processed_table))], ignore_index=True)
processed_table

In [ ]:
response.json()["result"]